In [1]:
import pandas as pd

In [2]:
pwd

'D:\\Kaggle-song-popularity\\Scripts'

D:\Kaggle-song-popularity


In [4]:
nn1 = pd.read_csv('submission_files/imputed_blends/spp-1-v2-test_iterative_impu_spp-BLEND-NN-1-0.5879.csv')
nn2 = pd.read_csv('submission_files/imputed_blends/spp-1-v2-test_iterative_impu_spp-BLEND-NN-2-0.5876.csv')
xgb_pred = pd.read_csv('submission_files/imputed_blends/spp-1-v2-test_iterative_impu_spp-BLEND-XGB-1-0.5864.csv')

In [6]:
xgb_pred

,id,song_popularity
0,0.0,0.392934
1,1.0,0.474245
2,2.0,0.251814
3,3.0,0.277137
4,4.0,0.380438
...,...,...
9995,9995.0,0.346843
9996,9996.0,0.292488
9997,9997.0,0.360433
9998,9998.0,0.442600


In [7]:
submission = pd.read_csv('csv/sample_submission.csv')

In [8]:
submission.song_popularity = nn1.song_popularity * 0.50 + nn2.song_popularity * 0.3 + xgb_pred.song_popularity * 0.2

In [11]:
submission=submission.drop(['id'],axis=1)

In [14]:
submission.to_csv('submission_files/imputed_blends/imputed_blend_nn1_nn2_xgb.csv', index = True)

In [15]:
submission

,song_popularity
0,0.374256
1,0.463153
2,0.229708
3,0.266924
4,0.350627
...,...
9995,0.327205
9996,0.281810
9997,0.334879
9998,0.426147


In [ ]:
train_base = pd.read_csv('https://raw.githubusercontent.com/mainadwitiya/Kaggle-song-popularity/main/csv/train.csv?token=GHSAT0AAAAAABPK7AEBVULXIUG5MRGDGVFGYPWRUNQ')
test_base = pd.read_csv('https://raw.githubusercontent.com/mainadwitiya/Kaggle-song-popularity/main/csv/test.csv?token=GHSAT0AAAAAABPK7AEA6PG33UFDPGL462I4YPWRVWQ')
submission = pd.read_csv('https://raw.githubusercontent.com/mainadwitiya/Kaggle-song-popularity/main/csv/sample_submission.csv?token=GHSAT0AAAAAABPK7AEATANZGA77FJZG6ZX6YPWZEGA')

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')


train = pd.DataFrame(imputer.fit_transform(train_base), columns = train_base.columns)
test = pd.DataFrame(imputer.fit_transform(test_base), columns = test_base.columns)

In [ ]:
#creating folds
from sklearn.model_selection import StratifiedKFold

train['kfold'] = -1

y_train = train_base.song_popularity
X_train = train.copy()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(skf.split(X_train,y_train )):
    train.loc[valid_indicies, "kfold"] = fold
train.to_csv("song_popularity_S5Fold.csv", index=False)

In [ ]:
df = pd.read_csv("./song_popularity_S5Fold.csv")

#features taken to train
features = [f for f in df.columns if f not in("id","kfold","song_popularity")]
test= test[features]

In [ ]:
def objective(trial): 
  for fold in range (5):
    # parameters
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 64),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 100,1000, step = 100),
        'max_depth' : trial.suggest_int('max_depth', 100,500, step = 100),
    }
    score = []

    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    ytrain = xtrain.song_popularity
    yvalid = xvalid.song_popularity
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    lE = StandardScaler()
    xtrain[features] = lE.fit_transform(xtrain[features])
    xvalid[features] = lE.transform(xvalid[features])
    xtest[features] = lE.transform(xtest[features])
    
    model= lgb.LGBMRegressor(**param, verbose = -1)
    model.fit(xtrain,ytrain,early_stopping_rounds=10,eval_set=[(xvalid,yvalid)],verbose=False)
    preds_valid = model.predict(xvalid)

    
    roc1= roc_auc_score(yvalid,preds_valid)
    #Score 
    score.append(roc1)
    print(f"fold|split:{fold},roc:{roc1}")
  
  return np.mean(score)


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2022-01-26 11:56:43,400] A new study created in memory with name: no-name-70b22bf6-db16-464f-8cf0-6266cf047d1e


fold|split:0,roc:0.5735903702241989
fold|split:1,roc:0.5724853814484804
fold|split:2,roc:0.5753097513792121
fold|split:3,roc:0.5722004145647491


[I 2022-01-26 11:56:44,930] Trial 0 finished with value: 0.5665576881774392 and parameters: {'lambda_l1': 0.01536227454204362, 'lambda_l2': 5.410292970203162e-06, 'num_leaves': 37, 'feature_fraction': 0.6731581338291512, 'bagging_fraction': 0.7854211956050916, 'bagging_freq': 3, 'min_child_samples': 46, 'min_data_in_leaf': 600, 'max_depth': 500}. Best is trial 0 with value: 0.5665576881774392.


fold|split:4,roc:0.5665576881774392
fold|split:0,roc:0.5748024239725692
fold|split:1,roc:0.5759192189046922
fold|split:2,roc:0.5777928896579756
fold|split:3,roc:0.5753292484032174


[I 2022-01-26 11:56:46,687] Trial 1 finished with value: 0.5701837389739888 and parameters: {'lambda_l1': 0.0006146614260020327, 'lambda_l2': 1.934168570142215e-07, 'num_leaves': 8, 'feature_fraction': 0.44702817503172426, 'bagging_fraction': 0.9180824421880464, 'bagging_freq': 5, 'min_child_samples': 54, 'min_data_in_leaf': 1000, 'max_depth': 200}. Best is trial 1 with value: 0.5701837389739888.


...

fold|split:4,roc:0.5666547543113822
fold|split:0,roc:0.5741595281585263
fold|split:1,roc:0.5745869447522478
fold|split:2,roc:0.5775149390043363


[I 2022-01-26 11:57:51,657] Trial 48 finished with value: 0.5642018614033213 and parameters: {'lambda_l1': 0.01131996569617726, 'lambda_l2': 0.039491643743949666, 'num_leaves': 4, 'feature_fraction': 0.4593897364618621, 'bagging_fraction': 0.40150051880167814, 'bagging_freq': 6, 'min_child_samples': 76, 'min_data_in_leaf': 1000, 'max_depth': 100}. Best is trial 22 with value: 0.5716472858177407.


fold|split:3,roc:0.5722117822067729
fold|split:4,roc:0.5642018614033213
fold|split:0,roc:0.5778324234159933
fold|split:1,roc:0.5777436411198309
fold|split:2,roc:0.5775466469672514
fold|split:3,roc:0.5763654916167856


[I 2022-01-26 11:57:53,305] Trial 49 finished with value: 0.5694445118983249 and parameters: {'lambda_l1': 9.043187748159588e-05, 'lambda_l2': 0.0012729957674815218, 'num_leaves': 17, 'feature_fraction': 0.6325244887781862, 'bagging_fraction': 0.9954783448215189, 'bagging_freq': 5, 'min_child_samples': 83, 'min_data_in_leaf': 900, 'max_depth': 500}. Best is trial 22 with value: 0.5716472858177407.


fold|split:4,roc:0.5694445118983249


In [ ]:
study.best_params

{'bagging_fraction': 0.5087313302209575,
 'bagging_freq': 2,
 'feature_fraction': 0.4814612171746773,
 'lambda_l1': 0.0002327135385401103,
 'lambda_l2': 5.850562790194026e-08,
 'max_depth': 400,
 'min_child_samples': 54,
 'min_data_in_leaf': 700,
 'num_leaves': 7}

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
prediction = []
score = []

#parameters = study.best_params

params={'alpha': 0.4275932000556265,
 'colsample_bytree': 0.3303272480369658,
 'eval_metric': 'mlogloss',
 'lambda': 0.013685373489365672,
 'learning_rate': 0.08882012188858979,
 'max_depth': 17,
 'min_child_weight': 223,
 'subsample': 0.5807927818586308,
 'n_estimators':5000,
 }

for fold in range (5):
    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = test.copy()
    
    ytrain = xtrain.song_popularity
    yvalid = xvalid.song_popularity
    
    xtrain = xtrain[features]
    xvalid = xvalid[features]
    
    lE = StandardScaler()
    xtrain[features] = lE.fit_transform(xtrain[features])
    xvalid[features] = lE.transform(xvalid[features])
    xtest[features] = lE.transform(xtest[features])
    
    model= GradientBoostingRegressor()
    model.fit(xtrain,ytrain)
    preds_valid = model.predict(xvalid)
    
    #Training model apply the test data and predict the output
    test_predict = model.predict(xtest)
    prediction.append(test_predict)
    roc1= roc_auc_score(yvalid,preds_valid)
    #Score 
    score.append(roc1)
    print(f"fold|split:{fold},roc:{roc1}")
    
print(np.mean(score),np.std(score))

fold|split:0,roc:0.5770355415905591
fold|split:1,roc:0.5737396675049038
fold|split:2,roc:0.5744024988573327
fold|split:3,roc:0.5681563000180465
fold|split:4,roc:0.5634788232589614
0.5713625662459607 0.004888926191848228


In [ ]:
#submission = pd.read_csv('sample_submission.csv')
submission['song_popularity']= model.predict(test)
submission.to_csv('lightgbreg_.csv',index=False)

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['song_popularity']= model.predict(test)
submission.to_csv('lgmb.csv',index=False)

FileNotFoundError: ignored

In [ ]:
submission.to_csv('cat_boost_tuned.csv',index=False)